In [43]:
%pip install jsonschema networkx numpy scipy pythreejs
%pip install compas>=2.8 --no-deps
%pip install compas_notebook --no-deps
%pip install compas_fd>=0.5.0 --no-deps

Note: you may need to restart the kernel to use updated packages.
zsh:1: 2.8 not found
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
zsh:1: 0.5.0 not found
Note: you may need to restart the kernel to use updated packages.


# Getting Started with COMPAS FD

Create a mesh from a grid of vertices and faces in the XY plane.

In [44]:
from compas.datastructures import Mesh

mesh = Mesh.from_meshgrid(dx=10, nx=10)

Move two opposite mesh corners out of the XY plane.

In [45]:
vertex = list(mesh.vertices_where(x=0, y=0))[0]
mesh.vertex_attribute(vertex, name='z', value=7)

vertex = list(mesh.vertices_where(x=10, y=10))[0]
mesh.vertex_attribute(vertex, name='z', value=7)

Extract the vertex coordinates.

In [46]:
vertices = mesh.vertices_attributes("xyz")

Identify the corners as fixed points, which will be able to provide reaction forces.

In [47]:
fixed = list(mesh.vertices_where(vertex_degree=2))

Create a list of "zero" loads.

In [48]:
loads = [[0, 0, 0] for _ in range(len(vertices))]

Create a list of edges, with each edge defined as a pair of vertex indices.

In [49]:
edges = list(mesh.edges())

Assign force densities to the edges.
Use `q=10.0` for edges on the boundary, and `q=1.0` everywhere else.

In [50]:
q = []
for edge in edges:
    if mesh.is_edge_on_boundary(edge):
        q.append(10)
    else:
        q.append(1.0)

Compute equilibrium with the base force density solver of `compas_fd`.
The result is stored in a `Result` data class.

In [51]:
from compas_fd.solvers import fd_numpy

result = fd_numpy(
    vertices=vertices,
    fixed=fixed,
    edges=edges,
    forcedensities=q,
    loads=loads,
)

Update the original mesh geometry using the vertex coordinates stored in the result.

In [52]:
for vertex, attr in mesh.vertices(data=True):
    attr["x"] = result.vertices[vertex, 0]
    attr["y"] = result.vertices[vertex, 1]
    attr["z"] = result.vertices[vertex, 2]

Visualize the equilibrium geometry and mark the fixed nodes.

In [59]:
from compas.colors import Color
from compas_notebook import Viewer
from compas_notebook.config import Config

config = Config()
config.view.camera.position = [15, -10, 3]
config.view.camera.target = [5, 5, 3]
config.view.camera.fov = 35

viewer = Viewer(config=config)

viewer.scene.add(mesh, color=Color.from_hex('#cccccc'), show_edges=True)

for vertex in fixed:
    viewer.scene.add(mesh.vertex_point(vertex), color=Color.red())

viewer.show()

# this is a temp hack until notebook is fixed
viewer.camera3.position = [15, -10, 3]
viewer.camera3.lookAt([5, 5, 3])
viewer.controls3.target = [5, 5, 3]
